https://www.sciencedirect.com/science/article/pii/S2211812814005318

In [841]:
# import dependencies
%matplotlib inline
from keras.models import Sequential
from keras.activations import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import *
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import talos as ta
import warnings
warnings.filterwarnings("ignore")

# suppres NumPy arrays scientific notation and round decimals to three places
np.set_printoptions(suppress=True)
np.printoptions(precision=3, suppress=True)

In [842]:
# import training data
train_data = pd.read_csv('Training_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
train_data.drop(columns=['Sr._No.'], inplace=True)
# import validation data
test_data = pd.read_csv('Test_Features_and_Labels.txt', delim_whitespace=True, encoding='ISO-8859-1')
test_data.drop(columns=['Sr._No.'], inplace=True)
# segregate a single row for validation
validation_data = test_data.loc[0]
test_data = test_data.iloc[1:]

In [843]:
# create NumPy arrays of training features and labels
train_features = train_data[['MRR(mm3/min)', 'Ra(µm)']].values
train_labels = train_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Training Features:\n\n', training_features)
print('\nTraining Labels:\n\n', training_labels) 

Training Features:

 [[ 306.67    0.36]
 [ 609.76    0.47]
 [ 909.28    0.52]
 [ 582.94    0.8 ]
 [1158.73    0.9 ]
 [1727.36    1.1 ]
 [ 943.34    1.63]
 [1875.96    1.76]
 [2797.84    2.17]
 [ 793.04    0.38]
 [1577.02    0.48]
 [2351.92    0.54]
 [1555.25    0.81]
 [3092.37    0.82]
 [4611.35    0.97]
 [2196.85    1.92]
 [4366.5     1.91]
 [6508.93    2.03]
 [ 875.13    0.29]
 [1735.95    0.37]
 [2582.46    0.38]
 [1745.24    0.82]
 [3461.88    0.79]
 [5149.9     1.11]
 [2549.2     1.75]
 [5055.49    1.89]
 [7518.86    1.82]]

Training Labels:

 [[ 280.        0.0508    0.4   ]
 [ 280.        0.0508    0.8   ]
 [ 280.        0.0508    1.2   ]
 [ 280.        0.1016    0.4   ]
 [ 280.        0.1016    0.8   ]
 [ 280.        0.1016    1.2   ]
 [ 280.        0.1524    0.4   ]
 [ 280.        0.1524    0.8   ]
 [ 280.        0.1524    1.2   ]
 [ 710.        0.0508    0.4   ]
 [ 710.        0.0508    0.8   ]
 [ 710.        0.0508    1.2   ]
 [ 710.        0.1016    0.4   ]
 [ 710.        0

In [844]:
# create NumPy arrays of validation features and labels
test_features = test_data[['MRR(mm3/min)', 'Ra(µm)']].values
test_labels = test_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values
print('Testing Features:\n\n', test_features)
print('\nTesting Labels:\n\n', test_labels)

Testing Features:

 [[2020.44    0.29]
 [6054.12    1.87]
 [5329.21    1.54]
 [5197.29    1.45]]

Testing Labels:

 [[1100.      0.05    0.69]
 [1095.      0.15    1.2 ]
 [1097.      0.13    1.14]
 [1094.      0.12    1.15]]


In [845]:
# create NumPy array for validation data
validation_features = validation_data[['MRR(mm3/min)', 'Ra(µm)']].values
validation_labels = validation_data[['Spindle_speed(rpm)', 'Feed_rate(mm/rev)', 'Depth_of_cut(mm)']].values

# reshape the validation data because it's only one row
validation_features = validation_features.reshape(1, -1)
validation_labels = validation_labels.reshape(1, -1)

print('Validation Features:\n\n', validation_features)
print('\nValidation Labels:\n\n', validation_labels)

Validation Features:

 [[1106.51    0.21]]

Validation Labels:

 [[1100.      0.05    0.4 ]]


In [846]:
# Transforms features by scaling each feature to a given range, -1 and 1 in this case
MinMax_scaler = MinMaxScaler(feature_range=(-1, 1))
train_features_MinMax = MinMax_scaler.fit_transform(train_features)
train_labels_MinMax = MinMax_scaler.fit_transform(train_labels)
test_features_MinMax = MinMax_scaler.fit_transform(test_features)
test_labels_MinMax = MinMax_scaler.fit_transform(test_labels)
validation_features_MinMax = MinMax_scaler.fit_transform(validation_features)
validation_labels_MinMax = MinMax_scaler.fit_transform(validation_labels)

#MinMax_scaler.inverse_transform(validation_labels_MinMax)

In [848]:
# Standardize features by removing the mean and scaling to unit variance
Standard_scaler = StandardScaler()
train_features_Standard = Standard_scaler.fit_transform(train_features)
train_labels_Standard = Standard_scaler.fit_transform(train_labels)
test_features_Standard = Standard_scaler.fit_transform(test_features)
test_labels_Standard = Standard_scaler.fit_transform(test_labels)
validation_features_Standard = Standard_scaler.fit_transform(validation_features)
validation_labels_Standard = Standard_scaler.fit_transform(validation_labels)

# Standard_scaler.inverse_transform(train_features_Standard)

In [850]:
# Define the MinMax model
MinMax_model = Sequential()
MinMax_model.add(Dense(50, input_dim=2, activation='relu'))
MinMax_model.add(Dense(100, activation='relu'))
MinMax_model.add(Dense(30, activation='relu'))
MinMax_model.add(Dense(3, activation='linear'))
MinMax_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

MinMax_model.fit(train_features_MinMax,
                 train_labels_MinMax,
                 validation_data=(test_features_MinMax, test_labels_MinMax),
                 epochs=500)

Train on 27 samples, validate on 4 samples
Epoch 1/500
27/27 [==============================] - 8s 314ms/step - loss: 0.6612 - acc: 0.4074 - val_loss: 0.7022 - val_acc: 0.2500
Epoch 2/500
27/27 [==============================] - 0s 870us/step - loss: 0.6481 - acc: 0.4074 - val_loss: 0.6829 - val_acc: 0.2500
Epoch 3/500
27/27 [==============================] - 0s 568us/step - loss: 0.6338 - acc: 0.4074 - val_loss: 0.6637 - val_acc: 0.2500
Epoch 4/500
27/27 [==============================] - 0s 537us/step - loss: 0.6195 - acc: 0.4074 - val_loss: 0.6433 - val_acc: 0.2500
Epoch 5/500
27/27 [==============================] - 0s 459us/step - loss: 0.6054 - acc: 0.4074 - val_loss: 0.6234 - val_acc: 0.2500
Epoch 6/500
27/27 [==============================] - 0s 589us/step - loss: 0.5923 - acc: 0.4074 - val_loss: 0.6054 - val_acc: 0.2500
Epoch 7/500
27/27 [==============================] - 0s 431us/step - loss: 0.5802 - acc: 0.4074 - val_loss: 0.5882 - val_acc: 0.2500
Epoch 8/500
27/27 [=======

Epoch 62/500
27/27 [==============================] - 0s 881us/step - loss: 0.2642 - acc: 0.5185 - val_loss: 0.3640 - val_acc: 0.7500
Epoch 63/500
27/27 [==============================] - 0s 716us/step - loss: 0.2623 - acc: 0.5556 - val_loss: 0.3713 - val_acc: 0.7500
Epoch 64/500
27/27 [==============================] - 0s 498us/step - loss: 0.2605 - acc: 0.5556 - val_loss: 0.3785 - val_acc: 0.7500
Epoch 65/500
27/27 [==============================] - 0s 451us/step - loss: 0.2588 - acc: 0.5556 - val_loss: 0.3857 - val_acc: 0.7500
Epoch 66/500
27/27 [==============================] - 0s 583us/step - loss: 0.2571 - acc: 0.5556 - val_loss: 0.3931 - val_acc: 0.7500
Epoch 67/500
27/27 [==============================] - 0s 911us/step - loss: 0.2555 - acc: 0.5556 - val_loss: 0.4007 - val_acc: 0.7500
Epoch 68/500
27/27 [==============================] - 0s 476us/step - loss: 0.2541 - acc: 0.5556 - val_loss: 0.4081 - val_acc: 0.7500
Epoch 69/500
27/27 [==============================] - 0s 569us

Epoch 123/500
27/27 [==============================] - 0s 597us/step - loss: 0.2190 - acc: 0.6296 - val_loss: 0.5712 - val_acc: 0.5000
Epoch 124/500
27/27 [==============================] - 0s 822us/step - loss: 0.2185 - acc: 0.6296 - val_loss: 0.5721 - val_acc: 0.5000
Epoch 125/500
27/27 [==============================] - 0s 1ms/step - loss: 0.2179 - acc: 0.6296 - val_loss: 0.5732 - val_acc: 0.5000
Epoch 126/500
27/27 [==============================] - 0s 526us/step - loss: 0.2172 - acc: 0.6296 - val_loss: 0.5737 - val_acc: 0.5000
Epoch 127/500
27/27 [==============================] - 0s 411us/step - loss: 0.2165 - acc: 0.6296 - val_loss: 0.5744 - val_acc: 0.5000
Epoch 128/500
27/27 [==============================] - 0s 835us/step - loss: 0.2159 - acc: 0.6296 - val_loss: 0.5741 - val_acc: 0.5000
Epoch 129/500
27/27 [==============================] - 0s 737us/step - loss: 0.2152 - acc: 0.6296 - val_loss: 0.5745 - val_acc: 0.5000
Epoch 130/500
27/27 [==============================] - 0s

Epoch 184/500
27/27 [==============================] - 0s 690us/step - loss: 0.1671 - acc: 0.6296 - val_loss: 0.6294 - val_acc: 0.5000
Epoch 185/500
27/27 [==============================] - 0s 510us/step - loss: 0.1662 - acc: 0.6296 - val_loss: 0.6295 - val_acc: 0.5000
Epoch 186/500
27/27 [==============================] - 0s 491us/step - loss: 0.1653 - acc: 0.6296 - val_loss: 0.6300 - val_acc: 0.5000
Epoch 187/500
27/27 [==============================] - 0s 828us/step - loss: 0.1642 - acc: 0.6296 - val_loss: 0.6340 - val_acc: 0.2500
Epoch 188/500
27/27 [==============================] - 0s 569us/step - loss: 0.1633 - acc: 0.6296 - val_loss: 0.6390 - val_acc: 0.0000e+00
Epoch 189/500
27/27 [==============================] - 0s 921us/step - loss: 0.1623 - acc: 0.6296 - val_loss: 0.6360 - val_acc: 0.2500
Epoch 190/500
27/27 [==============================] - 0s 922us/step - loss: 0.1614 - acc: 0.6296 - val_loss: 0.6365 - val_acc: 0.2500
Epoch 191/500
27/27 [==============================

27/27 [==============================] - 0s 354us/step - loss: 0.1167 - acc: 0.6667 - val_loss: 0.7318 - val_acc: 0.0000e+00
Epoch 244/500
27/27 [==============================] - 0s 429us/step - loss: 0.1159 - acc: 0.6667 - val_loss: 0.7253 - val_acc: 0.0000e+00
Epoch 245/500
27/27 [==============================] - 0s 573us/step - loss: 0.1154 - acc: 0.6667 - val_loss: 0.7264 - val_acc: 0.0000e+00
Epoch 246/500
27/27 [==============================] - 0s 598us/step - loss: 0.1147 - acc: 0.6667 - val_loss: 0.7240 - val_acc: 0.0000e+00
Epoch 247/500
27/27 [==============================] - 0s 472us/step - loss: 0.1140 - acc: 0.6296 - val_loss: 0.7255 - val_acc: 0.0000e+00
Epoch 248/500
27/27 [==============================] - 0s 618us/step - loss: 0.1133 - acc: 0.5926 - val_loss: 0.7349 - val_acc: 0.0000e+00
Epoch 249/500
27/27 [==============================] - 0s 583us/step - loss: 0.1126 - acc: 0.5926 - val_loss: 0.7314 - val_acc: 0.0000e+00
Epoch 250/500
27/27 [====================

Epoch 302/500
27/27 [==============================] - 0s 587us/step - loss: 0.0767 - acc: 0.6667 - val_loss: 0.7697 - val_acc: 0.0000e+00
Epoch 303/500
27/27 [==============================] - 0s 708us/step - loss: 0.0759 - acc: 0.6667 - val_loss: 0.7761 - val_acc: 0.0000e+00
Epoch 304/500
27/27 [==============================] - 0s 720us/step - loss: 0.0750 - acc: 0.7037 - val_loss: 0.7835 - val_acc: 0.0000e+00
Epoch 305/500
27/27 [==============================] - 0s 445us/step - loss: 0.0744 - acc: 0.7407 - val_loss: 0.7756 - val_acc: 0.0000e+00
Epoch 306/500
27/27 [==============================] - 0s 504us/step - loss: 0.0739 - acc: 0.7407 - val_loss: 0.7651 - val_acc: 0.0000e+00
Epoch 307/500
27/27 [==============================] - 0s 500us/step - loss: 0.0732 - acc: 0.6667 - val_loss: 0.7745 - val_acc: 0.0000e+00
Epoch 308/500
27/27 [==============================] - 0s 691us/step - loss: 0.0727 - acc: 0.6667 - val_loss: 0.7775 - val_acc: 0.0000e+00
Epoch 309/500
27/27 [======

Epoch 362/500
27/27 [==============================] - 0s 840us/step - loss: 0.0414 - acc: 0.7407 - val_loss: 0.7947 - val_acc: 0.2500
Epoch 363/500
27/27 [==============================] - 0s 751us/step - loss: 0.0412 - acc: 0.7778 - val_loss: 0.8138 - val_acc: 0.2500
Epoch 364/500
27/27 [==============================] - 0s 691us/step - loss: 0.0409 - acc: 0.8148 - val_loss: 0.8131 - val_acc: 0.2500
Epoch 365/500
27/27 [==============================] - 0s 580us/step - loss: 0.0405 - acc: 0.8148 - val_loss: 0.8125 - val_acc: 0.2500
Epoch 366/500
27/27 [==============================] - 0s 407us/step - loss: 0.0398 - acc: 0.7407 - val_loss: 0.8116 - val_acc: 0.2500
Epoch 367/500
27/27 [==============================] - 0s 452us/step - loss: 0.0392 - acc: 0.7407 - val_loss: 0.8102 - val_acc: 0.2500
Epoch 368/500
27/27 [==============================] - 0s 528us/step - loss: 0.0389 - acc: 0.7778 - val_loss: 0.8154 - val_acc: 0.2500
Epoch 369/500
27/27 [==============================] - 

Epoch 423/500
27/27 [==============================] - 0s 886us/step - loss: 0.0220 - acc: 0.7778 - val_loss: 0.8370 - val_acc: 0.2500
Epoch 424/500
27/27 [==============================] - 0s 711us/step - loss: 0.0218 - acc: 0.7407 - val_loss: 0.8414 - val_acc: 0.2500
Epoch 425/500
27/27 [==============================] - 0s 641us/step - loss: 0.0217 - acc: 0.7407 - val_loss: 0.8560 - val_acc: 0.2500
Epoch 426/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0216 - acc: 0.8148 - val_loss: 0.8438 - val_acc: 0.2500
Epoch 427/500
27/27 [==============================] - 0s 579us/step - loss: 0.0214 - acc: 0.7407 - val_loss: 0.8494 - val_acc: 0.2500
Epoch 428/500
27/27 [==============================] - 0s 462us/step - loss: 0.0211 - acc: 0.7778 - val_loss: 0.8559 - val_acc: 0.2500
Epoch 429/500
27/27 [==============================] - 0s 426us/step - loss: 0.0209 - acc: 0.8148 - val_loss: 0.8432 - val_acc: 0.2500
Epoch 430/500
27/27 [==============================] - 0s

Epoch 484/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0150 - acc: 0.7407 - val_loss: 0.8624 - val_acc: 0.2500
Epoch 485/500
27/27 [==============================] - 0s 456us/step - loss: 0.0150 - acc: 0.8889 - val_loss: 0.8547 - val_acc: 0.2500
Epoch 486/500
27/27 [==============================] - 0s 761us/step - loss: 0.0149 - acc: 0.7407 - val_loss: 0.8566 - val_acc: 0.2500
Epoch 487/500
27/27 [==============================] - 0s 599us/step - loss: 0.0148 - acc: 0.8519 - val_loss: 0.8534 - val_acc: 0.2500
Epoch 488/500
27/27 [==============================] - 0s 518us/step - loss: 0.0147 - acc: 0.7407 - val_loss: 0.8583 - val_acc: 0.2500
Epoch 489/500
27/27 [==============================] - 0s 694us/step - loss: 0.0147 - acc: 0.8148 - val_loss: 0.8577 - val_acc: 0.2500
Epoch 490/500
27/27 [==============================] - 0s 609us/step - loss: 0.0146 - acc: 0.7778 - val_loss: 0.8609 - val_acc: 0.2500
Epoch 491/500
27/27 [==============================] - 0s

In [851]:
# Define the Standard model
Standard_model = Sequential()
Standard_model.add(Dense(50, input_dim=2, activation='relu'))
Standard_model.add(Dense(100, activation='relu'))
Standard_model.add(Dense(30, activation='relu'))
Standard_model.add(Dense(3, activation='linear'))
Standard_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Standard_model.fit(train_features_Standard,
                   train_labels_Standard,
                   validation_data=(test_features_Standard, test_labels_Standard),
                   epochs=500)

Train on 27 samples, validate on 4 samples
Epoch 1/500
27/27 [==============================] - 9s 327ms/step - loss: 1.1516 - acc: 0.2963 - val_loss: 0.9405 - val_acc: 0.2500
Epoch 2/500
27/27 [==============================] - 0s 427us/step - loss: 1.1103 - acc: 0.2963 - val_loss: 0.9109 - val_acc: 0.2500
Epoch 3/500
27/27 [==============================] - 0s 576us/step - loss: 1.0656 - acc: 0.2963 - val_loss: 0.8812 - val_acc: 0.2500
Epoch 4/500
27/27 [==============================] - 0s 423us/step - loss: 1.0226 - acc: 0.2963 - val_loss: 0.8516 - val_acc: 0.2500
Epoch 5/500
27/27 [==============================] - 0s 782us/step - loss: 0.9818 - acc: 0.2963 - val_loss: 0.8231 - val_acc: 0.2500
Epoch 6/500
27/27 [==============================] - 0s 743us/step - loss: 0.9434 - acc: 0.3333 - val_loss: 0.7955 - val_acc: 0.2500
Epoch 7/500
27/27 [==============================] - 0s 409us/step - loss: 0.9079 - acc: 0.4444 - val_loss: 0.7689 - val_acc: 0.5000
Epoch 8/500
27/27 [=======

Epoch 62/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3678 - acc: 0.6667 - val_loss: 0.7193 - val_acc: 1.0000
Epoch 63/500
27/27 [==============================] - 0s 933us/step - loss: 0.3656 - acc: 0.6296 - val_loss: 0.7294 - val_acc: 1.0000
Epoch 64/500
27/27 [==============================] - 0s 542us/step - loss: 0.3635 - acc: 0.6296 - val_loss: 0.7394 - val_acc: 1.0000
Epoch 65/500
27/27 [==============================] - 0s 876us/step - loss: 0.3615 - acc: 0.6296 - val_loss: 0.7486 - val_acc: 1.0000
Epoch 66/500
27/27 [==============================] - 0s 641us/step - loss: 0.3598 - acc: 0.6296 - val_loss: 0.7569 - val_acc: 1.0000
Epoch 67/500
27/27 [==============================] - 0s 648us/step - loss: 0.3580 - acc: 0.6296 - val_loss: 0.7639 - val_acc: 1.0000
Epoch 68/500
27/27 [==============================] - 0s 610us/step - loss: 0.3565 - acc: 0.6296 - val_loss: 0.7688 - val_acc: 1.0000
Epoch 69/500
27/27 [==============================] - 0s 574us/s

Epoch 123/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3133 - acc: 0.6667 - val_loss: 0.9221 - val_acc: 1.0000
Epoch 124/500
27/27 [==============================] - 0s 541us/step - loss: 0.3125 - acc: 0.6667 - val_loss: 0.9210 - val_acc: 1.0000
Epoch 125/500
27/27 [==============================] - 0s 542us/step - loss: 0.3117 - acc: 0.6667 - val_loss: 0.9212 - val_acc: 1.0000
Epoch 126/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3108 - acc: 0.6667 - val_loss: 0.9256 - val_acc: 1.0000
Epoch 127/500
27/27 [==============================] - 0s 953us/step - loss: 0.3100 - acc: 0.6667 - val_loss: 0.9332 - val_acc: 1.0000
Epoch 128/500
27/27 [==============================] - 0s 1ms/step - loss: 0.3091 - acc: 0.6667 - val_loss: 0.9387 - val_acc: 1.0000
Epoch 129/500
27/27 [==============================] - 0s 509us/step - loss: 0.3083 - acc: 0.6667 - val_loss: 0.9422 - val_acc: 1.0000
Epoch 130/500
27/27 [==============================] - 0s 494

Epoch 184/500
27/27 [==============================] - 0s 578us/step - loss: 0.2489 - acc: 0.8148 - val_loss: 1.1747 - val_acc: 0.2500
Epoch 185/500
27/27 [==============================] - 0s 511us/step - loss: 0.2477 - acc: 0.8148 - val_loss: 1.1740 - val_acc: 0.2500
Epoch 186/500
27/27 [==============================] - 0s 595us/step - loss: 0.2464 - acc: 0.8148 - val_loss: 1.1811 - val_acc: 0.2500
Epoch 187/500
27/27 [==============================] - 0s 476us/step - loss: 0.2451 - acc: 0.8148 - val_loss: 1.1946 - val_acc: 0.2500
Epoch 188/500
27/27 [==============================] - 0s 401us/step - loss: 0.2438 - acc: 0.8148 - val_loss: 1.2018 - val_acc: 0.2500
Epoch 189/500
27/27 [==============================] - 0s 433us/step - loss: 0.2425 - acc: 0.8148 - val_loss: 1.2032 - val_acc: 0.2500
Epoch 190/500
27/27 [==============================] - 0s 598us/step - loss: 0.2413 - acc: 0.8148 - val_loss: 1.2012 - val_acc: 0.2500
Epoch 191/500
27/27 [==============================] - 

Epoch 245/500
27/27 [==============================] - 0s 816us/step - loss: 0.1573 - acc: 0.8889 - val_loss: 1.6406 - val_acc: 0.2500
Epoch 246/500
27/27 [==============================] - 0s 563us/step - loss: 0.1555 - acc: 0.8889 - val_loss: 1.6522 - val_acc: 0.2500
Epoch 247/500
27/27 [==============================] - 0s 521us/step - loss: 0.1536 - acc: 0.8889 - val_loss: 1.6743 - val_acc: 0.2500
Epoch 248/500
27/27 [==============================] - 0s 516us/step - loss: 0.1519 - acc: 0.8889 - val_loss: 1.6930 - val_acc: 0.2500
Epoch 249/500
27/27 [==============================] - 0s 475us/step - loss: 0.1501 - acc: 0.8889 - val_loss: 1.6943 - val_acc: 0.2500
Epoch 250/500
27/27 [==============================] - 0s 1ms/step - loss: 0.1484 - acc: 0.8889 - val_loss: 1.6944 - val_acc: 0.2500
Epoch 251/500
27/27 [==============================] - 0s 493us/step - loss: 0.1465 - acc: 0.8889 - val_loss: 1.7055 - val_acc: 0.2500
Epoch 252/500
27/27 [==============================] - 0s

27/27 [==============================] - 0s 546us/step - loss: 0.0733 - acc: 0.8519 - val_loss: 2.1734 - val_acc: 0.0000e+00
Epoch 305/500
27/27 [==============================] - 0s 1ms/step - loss: 0.0724 - acc: 0.8519 - val_loss: 2.1644 - val_acc: 0.2500
Epoch 306/500
27/27 [==============================] - 0s 299us/step - loss: 0.0712 - acc: 0.8519 - val_loss: 2.1721 - val_acc: 0.2500
Epoch 307/500
27/27 [==============================] - 0s 572us/step - loss: 0.0702 - acc: 0.8519 - val_loss: 2.2012 - val_acc: 0.0000e+00
Epoch 308/500
27/27 [==============================] - 0s 467us/step - loss: 0.0695 - acc: 0.8148 - val_loss: 2.1992 - val_acc: 0.2500
Epoch 309/500
27/27 [==============================] - 0s 623us/step - loss: 0.0685 - acc: 0.8148 - val_loss: 2.1808 - val_acc: 0.2500
Epoch 310/500
27/27 [==============================] - 0s 649us/step - loss: 0.0675 - acc: 0.8519 - val_loss: 2.1998 - val_acc: 0.2500
Epoch 311/500
27/27 [==============================] - 0s 461us

Epoch 365/500
27/27 [==============================] - 0s 646us/step - loss: 0.0326 - acc: 0.8889 - val_loss: 2.3232 - val_acc: 0.2500
Epoch 366/500
27/27 [==============================] - 0s 587us/step - loss: 0.0322 - acc: 0.8889 - val_loss: 2.3276 - val_acc: 0.2500
Epoch 367/500
27/27 [==============================] - 0s 616us/step - loss: 0.0318 - acc: 0.8889 - val_loss: 2.3271 - val_acc: 0.2500
Epoch 368/500
27/27 [==============================] - 0s 663us/step - loss: 0.0313 - acc: 0.8519 - val_loss: 2.3266 - val_acc: 0.2500
Epoch 369/500
27/27 [==============================] - 0s 389us/step - loss: 0.0309 - acc: 0.8519 - val_loss: 2.3288 - val_acc: 0.2500
Epoch 370/500
27/27 [==============================] - 0s 322us/step - loss: 0.0306 - acc: 0.8519 - val_loss: 2.3311 - val_acc: 0.2500
Epoch 371/500
27/27 [==============================] - 0s 329us/step - loss: 0.0302 - acc: 0.8889 - val_loss: 2.3307 - val_acc: 0.2500
Epoch 372/500
27/27 [==============================] - 

Epoch 426/500
27/27 [==============================] - 0s 491us/step - loss: 0.0171 - acc: 0.8519 - val_loss: 2.6835 - val_acc: 0.2500
Epoch 427/500
27/27 [==============================] - 0s 567us/step - loss: 0.0170 - acc: 0.8148 - val_loss: 2.6860 - val_acc: 0.2500
Epoch 428/500
27/27 [==============================] - 0s 471us/step - loss: 0.0169 - acc: 0.8519 - val_loss: 2.6936 - val_acc: 0.2500
Epoch 429/500
27/27 [==============================] - 0s 348us/step - loss: 0.0167 - acc: 0.8889 - val_loss: 2.7100 - val_acc: 0.2500
Epoch 430/500
27/27 [==============================] - 0s 483us/step - loss: 0.0166 - acc: 0.8148 - val_loss: 2.7037 - val_acc: 0.2500
Epoch 431/500
27/27 [==============================] - 0s 625us/step - loss: 0.0165 - acc: 0.8889 - val_loss: 2.7171 - val_acc: 0.2500
Epoch 432/500
27/27 [==============================] - 0s 478us/step - loss: 0.0164 - acc: 0.8519 - val_loss: 2.7034 - val_acc: 0.2500
Epoch 433/500
27/27 [==============================] - 

Epoch 487/500
27/27 [==============================] - 0s 559us/step - loss: 0.0119 - acc: 0.8889 - val_loss: 3.0944 - val_acc: 0.2500
Epoch 488/500
27/27 [==============================] - 0s 726us/step - loss: 0.0120 - acc: 0.8519 - val_loss: 3.0387 - val_acc: 0.2500
Epoch 489/500
27/27 [==============================] - 0s 537us/step - loss: 0.0121 - acc: 0.8519 - val_loss: 3.0998 - val_acc: 0.2500
Epoch 490/500
27/27 [==============================] - 0s 363us/step - loss: 0.0119 - acc: 0.8519 - val_loss: 3.0713 - val_acc: 0.2500
Epoch 491/500
27/27 [==============================] - 0s 802us/step - loss: 0.0117 - acc: 0.9259 - val_loss: 3.0772 - val_acc: 0.2500
Epoch 492/500
27/27 [==============================] - 0s 573us/step - loss: 0.0116 - acc: 0.8519 - val_loss: 3.1044 - val_acc: 0.2500
Epoch 493/500
27/27 [==============================] - 0s 591us/step - loss: 0.0116 - acc: 0.8889 - val_loss: 3.0776 - val_acc: 0.2500
Epoch 494/500
27/27 [==============================] - 

In [852]:
# MinMax prediction
prediction = MinMax_model.predict(validation_features_MinMax)
prediction = MinMax_scaler.inverse_transform(prediction)
prediction

array([[1100.4938    ,    0.06358229,    0.24108574]], dtype=float32)

In [853]:
# label to compare
MinMax_scaler.inverse_transform(validation_labels_MinMax)

array([[1100.  ,    0.05,    0.4 ]])

In [854]:
# Standard prediction
prediction = Standard_model.predict(validation_features_Standard)
prediction = Standard_scaler.inverse_transform(prediction)
prediction

array([[1100.4075    ,    0.43921053,   -0.22573748]], dtype=float32)

In [855]:
# label to compare
Standard_scaler.inverse_transform(validation_labels_Standard)

array([[1100.  ,    0.05,    0.4 ]])